In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.python.framework import ops
from tqdm import tqdm
import pickle
from tensorflow.core.protobuf import config_pb2

tf.reset_default_graph()

In [2]:
img_shape = 1024
# z_dim = 
x_input = tf.placeholder(tf.float32, (None, img_shape, img_shape, 3), name='Input')
x_input_bw = tf.placeholder(tf.float32, (None, img_shape, img_shape, 1), name='Input_bw')
x_input_mask = tf.placeholder(tf.float32, (None, img_shape, img_shape, 1), name='Input_mask')
x_target = tf.placeholder(tf.float32, (None, img_shape, img_shape, 3), name='Target')
# decoder_input = tf.placeholder(dtype=tf.float32, shape=[1, z_dim], name='Decoder_input')

In [3]:
def datanoisefree(i):
    training_data=[]
    training_data_bw=[]
    for j in range((i*1),(1+(i*1))):
        image = cv2.imread('D:\satellitedataCopy\shufflelabel\%d.jpg' %(j))
        new_array=cv2.resize(image,(img_shape,img_shape))
        norm_img = np.zeros((img_shape,img_shape))
        final_img = cv2.normalize(new_array,  norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        new_array_bw = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
        new_array_bw = cv2.resize(new_array_bw,(img_shape,img_shape))
        new_array_bw = np.reshape(new_array_bw, (img_shape,img_shape,1))
        training_data.append(new_array)
        training_data_bw.append(new_array_bw)
    return training_data, training_data_bw

In [4]:
# def data(i):
#     training_data=[]
#     training_data_bw=[]
#     for j in range((i*124),(124+(i*124))):
#         image = cv2.imread('D:\satellitedata\shufflenoisydata\%d.jpg' %(j))
#         new_array=cv2.resize(image,(512,512))
#         norm_img = np.zeros((512,512))
#         final_img = cv2.normalize(new_array,  norm_img, 0, 255, cv2.NORM_MINMAX)
#         new_array = final_img
#         new_array_bw = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
#         new_array_bw = cv2.resize(new_array_bw,(512,512))
#         new_array_bw = np.reshape(new_array_bw, (512,512,1))
#         training_data.append(new_array)
#         training_data_bw.append(new_array_bw)
#     return training_data, training_data_bw

In [5]:
def datafornoisyimages(i):
    training_data=[]
    training_data_bw=[]
    training_data_mask=[]
    for j in range((i*124),(124+(i*124))):
        image = cv2.imread('D:\satellitedata\shufflednoisedata\%d.jpg' %(j))
        mask_image = cv2.imread('D:\satellitedata\shufflemask\%d.jpg' %(j))
        
        new_array=cv2.resize(image,(img_shape,img_shape))
        norm_img = np.zeros((img_shape,img_shape))
        final_img = cv2.normalize(new_array,  norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        
        mask_array = cv2.normalize(mask_image,  norm_img, 0, 255, cv2.NORM_MINMAX)
        mask_array = cv2.resize(mask_array, (img_shape,img_shape)).reshape((img_shape,img_shape,1))
        
        new_array_bw = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
        new_array_bw = cv2.resize(new_array_bw,(img_shape,img_shape))
        new_array_bw = np.reshape(new_array_bw, (img_shape,img_shape,1))
        
        training_data.append(new_array)
        training_data_bw.append(new_array_bw)
        training_data_mask.append(mask_array)
    return training_data, training_data_bw, training_data_mask

In [6]:
# def label(i):
#     label = []
#     for j in range((i*124),(124+(i*124))):
#         image = cv2.imread('D:\\satellitedata\\shufflelabel\\%d.jpg' %(j))
#         new_array=cv2.resize(image,(512,512))
#         norm_img = np.zeros((512,512))
#         final_img = cv2.normalize(new_array, norm_img, 0, 255, cv2.NORM_MINMAX)
#         new_array = final_img
#         label.append(new_array)
#     return label

In [7]:
# The encoder network for noise-free(nf) images
def encoder_nf(z,z_bw):
    filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}
    conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv2 = tf.layers.conv2d(z_bw, filters[2], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv = tf.concat([conv1, conv2], 3)
    convshortcut = conv
    conv3 = tf.layers.conv2d(conv, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv4 = tf.layers.conv2d(conv3, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    convD1 = tf.layers.conv2d(conv4,filters[4],(5,5), strides=(1, 1), padding='valid',activation=tf.nn.relu,dilation_rate=(2, 2), use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    convD2 = tf.layers.conv2d(convD1,filters[4],(5,5), strides=(1, 1), padding='valid',activation=None,dilation_rate=(2, 2), use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv_shortcut1 = tf.layers.conv2d(convshortcut,filters[4],(79,79), strides=(4, 4), padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    print(np.shape(convD2))
    print(np.shape(convshortcut))
    print(np.shape(conv_shortcut1))
    latent_variable = tf.math.add(convD2, conv_shortcut1)
    # e_dense_1 = tf.nn.relu(dense(x, input_dim, n_l1, 'e_dense_1'))
    # e_dense_2 = tf.nn.relu(dense(e_dense_1, n_l1, n_l2, 'e_dense_2'))
    # latent_variable = dense(e_dense_2, n_l2, z_dim, 'e_latent_variable')
    return latent_variable

In [8]:
# The encoder network for noisy(n) images
def encoder_n(z,z_bw, z_mask):
    filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}
    conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(2, 2), padding='valid',activation=None, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv2 = tf.layers.conv2d(z_bw, filters[2], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv0 = tf.layers.conv2d(z_mask, filters[2], (3,3),strides=(2,2),padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv = tf.concat([conv1, conv2, conv0], 3)
    convshortcut = conv
    conv3 = tf.layers.conv2d(conv, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv4 = tf.layers.conv2d(conv3, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    convD1 = tf.layers.conv2d(conv4,filters[4],(5,5), strides=(1, 1), padding='valid',activation=tf.nn.relu,dilation_rate=(2, 2), use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    convD2 = tf.layers.conv2d(convD1,filters[4],(5,5), strides=(1, 1), padding='valid',activation=None,dilation_rate=(2, 2), use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv_shortcut1 = tf.layers.conv2d(convshortcut,filters[4],(79,79), strides=(4, 4), padding='valid',activation=None,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    print(np.shape(convD2))
    print(np.shape(conv_shortcut1))
    latent_variable = tf.math.add(convD2, conv_shortcut1)
    # e_dense_1 = tf.nn.relu(dense(x, input_dim, n_l1, 'e_dense_1'))
    # e_dense_2 = tf.nn.relu(dense(e_dense_1, n_l1, n_l2, 'e_dense_2'))
    # latent_variable = dense(e_dense_2, n_l2, z_dim, 'e_latent_variable')
    return latent_variable

In [9]:
def decoder_nf(x):
    act = tf.nn.relu(x)
    upsample1 = tf.image.resize_nearest_neighbor(act, (512,512))
    conv5 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv5, (636,636))
    conv6 = tf.layers.conv2d(upsample2, 20, (5,5), padding='same',activation=tf.nn.relu)
    upsample3 = tf.image.resize_nearest_neighbor(conv6, (776,776))
    conv7 = tf.layers.conv2d(upsample3, 3, (7,7), padding='same',activation=tf.nn.relu)
    upsample4 = tf.image.resize_nearest_neighbor(conv7, (1024,1024))
    output = tf.layers.conv2d(upsample4, 3, (7,7), padding='same',activation=tf.nn.relu)
    # d_dense_1 = tf.nn.relu(dense(x, z_dim, n_l2, 'd_dense_1'))
    # d_dense_2 = tf.nn.relu(dense(d_dense_1, n_l2, n_l1, 'd_dense_2'))
    # output = tf.nn.sigmoid(dense(d_dense_2, n_l1, input_dim, 'd_output'))
    return output

In [10]:
def decoder_n(x):
    act = tf.nn.relu(x)
    upsample1 = tf.image.resize_nearest_neighbor(act, (512,512))
    conv5 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv4, (636,636))
    conv6 = tf.layers.conv2d(upsample2, 20, (5,5), padding='same',activation=tf.nn.relu)
    upsample3 = tf.image.resize_nearest_neighbor(conv5, (776,776))
    conv7 = tf.layers.conv2d(upsample3, 3, (7,7), padding='same',activation=tf.nn.relu)
    upsample4 = tf.image.resize_nearest_neighbor(conv5, (1024,1024))
    output = tf.layers.conv2d(upsample4, 3, (7,7), padding='same',activation=tf.nn.relu)
    # d_dense_1 = tf.nn.relu(dense(x, z_dim, n_l2, 'd_dense_1'))
    # d_dense_2 = tf.nn.relu(dense(d_dense_1, n_l2, n_l1, 'd_dense_2'))
    # output = tf.nn.sigmoid(dense(d_dense_2, n_l1, input_dim, 'd_output'))
    return output

In [11]:
encoder_output = encoder_nf(x_input, x_input_bw)
decoder_output = decoder_nf(encoder_output)
print(np.shape(decoder_output))
print(np.shape(encoder_output))

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
(?, 109, 109, 60)
(?, 511, 511, 40)
(?, 109, 109, 60)
(?, 1024, 1024, 3)
(?, 109, 109, 60)


In [12]:
lr=0.001
autoencoder_loss = tf.reduce_mean(tf.square(x_target - decoder_output))
autoencoder_optimizer = tf.train.AdamOptimizer(learning_rate=lr,
                                                   beta1=0.9).minimize(autoencoder_loss)
   

In [13]:
batch_size=1
epochs=100
init=tf.global_variables_initializer()

In [ ]:
samples=[] 
minibatch_loss = 0
cost=[]
latentvars=[]
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(epochs)):
        num_batches=int(21080/1)
        for i in range(num_batches):
            X, X_bw = datanoisefree(i)
            Y, _ = datanoisefree(i)
            _, temp_loss = sess.run([autoencoder_optimizer,autoencoder_loss], feed_dict={x_input:X, x_input_bw:X_bw, x_target:Y})

            minibatch_loss += temp_loss / num_batches
            latentvars.append(encoder_output)
#             if (epoch==(epoch-1)):
#                 latent_vector = 
        print("on epoch{}".format(epoch))
        # Print the cost every epoch
        print ("Cost after epoch %i: %f" % (epoch, minibatch_loss))
        cost.append(minibatch_loss)
    #saving latent variables of noise free images using pickle
        with open('lv_nf.data', 'wb') as filehandle:
            pickle.dump(latentvars, filehandle)
        

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

In [ ]:
#weights variables for discriminator

In [ ]:
def discriminator(X,reuse=None):
    conv1 = tf.layers.conv2d(X, 20, (3,3),strides=(2, 2), padding='valid',activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    conv2 = tf.layers.conv2d(conv1,40, (3,3),strides=(2,2),padding='valid',activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
    flat = tf.contrib.layers.flatten(conv2)
    finallayer = tf.add(tf.matmul(flat,weights['disc_final']), bias['disc_final'])
#     logits=tf.layers.dense(flat,units=1)
#     output=tf.sigmoid(logits)
    output = finallayer
    return output
#     return output,logits

In [ ]:
encoder_output_n = encoder_n(x_input,x_input_bw, x_input_mask)
decoder_output_n = decoder(encoder_output_n)
d_real = discriminator(encoder_output)
d_fake = discriminator(encoder_output_n, reuse=True)

decoder_image = decoder(decoder_input, reuse=True)


In [ ]:
lr=0.01
autoencoder_loss = tf.reduce_mean(tf.square(x_target - decoder_output))
disc_loss = tf.reduce_mean(d_real)-tf.reduce_mean(d_fake)

autoencoder_optimizer = tf.train.AdamOptimizer(learning_rate=lr,
                                                   beta1=0.9).minimize(autoencoder_loss)
discriminator_optimizer = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(-disc_loss)

In [ ]:
batch_size=62
epochs=100
init=tf.global_variables_initializer()

In [ ]:
samples_n=[] 
minibatch_loss_n = 0
cost_n=[]
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm(range(epochs)):
        num_batches=
        for i in range(num_batches):
            X, X_bw, X_mask = datafornoisyimages(i)
            Y, _ = datafornoisyimages(i)
            _, temp_loss = sess.run([autoencoder_optimizer,autoencoder_loss], feed_dict={x_input:X, x_input_bw:X_bw, x_input_mask:X_mask, x_target:Y})
            _, temp_loss = sess.run([discriminator_optimizer,discriminator_loss], feed_dict={x_input:X, x_input_bw:X_bw, x_input_mask:X_mask})
            minibatch_loss_n += temp_loss / num_batches
            
        print("on epoch{}".format(epoch))
        # Print the cost every epoch
        print ("Cost after epoch %i: %f" % (epoch, minibatch_loss_n))
        cost_n.append(minibatch_loss_n)